# FEAT GROUP MODEL -- MULTIPLE COMPARISON CORRECTION AND REPORTING SCRIPT

**Date modified:** 10/25/2024<br>
**Authors:** Amy Hegarty, Intermountain Neuroimaging Consortium

**Description:** <br>
Work in prgress...
User should update files paths in first section below before proceeding. 

Sections:
1. **flywheel resolver** -- updated paths to local pl locations (skip if models were run outside flywheel)
2. **randomise_gfeat_grid3** -- must up to date randomise script from banich lab. Computes permutation based statistics and inital cluster correction thresholding
3. **randclustercontrasts** -- secondary scripts to store thresholded tstat maps, apply fdr, and generate cluster metrics
4. **mine4stats and overlays** - generates "*.summary" folders used for visualization
5. **pdf reports** (uncorrected zstats, fwe / fdr corrected tstats)
6. **upload to flywheel**
-----

Before starting...
1. Be sure you have configured your conda environment to view ics managed conda environments and packages. If you haven't get started [here](https://inc-documentation.readthedocs.io/en/latest/pl_and_blanca_basics.html#setting-up-conda-environments).

2. Be sure to select the `incenv` kernel from the list of availible kernels. If you don't see the `incenv` kernel, contact Amy Hegarty <Amy.Hegarty@colorado.edu> of follow the instructions [here](https://inc-documentation.readthedocs.io/en/latest/pl_and_blanca_basics.html#setting-up-conda-environments) to setup a new kernel in a shared conda environment. 


## SETUP PATHS
Edit this section !!
------------------------------

In [ ]:
path="/path/to/group/level/feat/models"    # path points to local folder location for gfeat models
gfeat_prefix=""                            # string to select which gfeat models to include... (useful in 3rd Level designs)
flywheel_path="group/project"              # Flywheel Group and Project (used for upload at the end)

### Helper functions and packages

In [ ]:
import glob
import pandas as pd
import re
from pathlib import Path
import os
import sys
import numpy as np
from datetime import date
import subprocess as sp
import logging
import time
from string import Template

# set default permissions
os.umask(0o002);

In [ ]:
def shell(cmd, workdir=None):
    terminal = sp.Popen(
            cmd, shell=True, stdout=sp.PIPE, stderr=sp.PIPE, universal_newlines=True, cwd=workdir
        )
    stdout, stderr = terminal.communicate()
    log.debug("\n %s", stdout)
    log.debug("\n %s", stderr)

    output = stdout.strip("\n").split("\n")

    return output

In [ ]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('main')

In [ ]:
from IPython.display import HTML, display

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4></td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))
    
    return HTML(html)

In [ ]:
def findstem(arr):
 
    # Determine size of the array
    n = len(arr)
 
    # Take first word from array
    # as reference
    s = arr[0]
    l = len(s)
 
    res = ""
 
    for i in range(l):
        for j in range(i + 1, l + 1):
 
            # generating all possible substrings
            # of our reference string arr[0] i.e s
            stem = s[i:j]
            k = 1
            for k in range(1, n):
 
                # Check if the generated stem is
                # common to all words
                if stem not in arr[k]:
                    break
 
            # If current substring is present in
            # all strings and its length is greater
            # than current result
            if (k + 1 == n and len(res) < len(stem)):
                res = stem
 
    return res

In [ ]:
def get_local_folder():
    return os.path.dirname(os.path.realpath(__file__))

In [ ]:
def set_compute_vars():
    alpine_queue={"qos":"normal","partition":"amilan","account":"ucb-general"}
    blanca_queue={"qos":"blanca-ics","partition":"blanca-ics","account":"blanca-ics"}
    
    if "bnode" in os.environ["HOSTNAME"]:
        return blanca_queue
    else:
        return alpine_queue

In [ ]:
def remove_files_by_wildcard(pattern):
    """Removes files matching the given wildcard pattern."""
    files = glob.glob(pattern)
    for file_path in files:
        try:
            os.remove(file_path)
            print(f"Removed: {file_path}")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error removing {file_path}: {e}")

### STEP 1: RESOLVE FLYWHEEL PATHS
If lower / intermediate models were run in flywheel, first resolve the lower level model paths. These needs to be run only once, but will allow mine4stats to correctly identify the lower level contrasts.

In [ ]:
#Update this!!
subject_feat_path = "/path/to/your/subjectlevel/featmodels/"

#change path to be automated from above...
cmd="""#!/bin/bash

umask g+w

export PATH=$PWD/bin:$PATH
# replace the filepath in lower level models so that fsfinfo works locally on pl!
for FSFDIR1 in `ls -d {FEATPATH}/sub-*/ses-*/*.gfeat/cope*.feat` ; do
	for MYVAR in `fsfinfo -i $FSFDIR1`; do
		if [[ $MYVAR == *"gear-temp"* ]]; then
			MYVARNEW="{FEATPATH}`echo $MYVAR | cut -d"/" -f12-`"
			echo "Replacing: $MYVAR --> $MYVARNEW"
			sed -i "s#$MYVAR#$MYVARNEW#g" "$FSFDIR1/design.fsf"
		fi
	done
done
"""
# writing batch file
with open("rm.rename_paths.sh","w+") as f:
    f.writelines(
        cmd.format(**{
           'FEATPATH': subject_feat_path,
        })
    )
    
shell(". ./rm.rename_paths.sh")

In [ ]:
# WARNING!! REMOVES FILES BY WILDCARD. BE SURE BEFORE RUNNING!
remove_files_by_wildcard("rm.*.sh")

### STEP 2: RADOMISE CLUSTER CORRECTION
Nonparametric permutation testing (fsl randomise) is used to estimate the statistics maps. Maps are thresholded using uncorrected and FWE corrected t-statistics. Cluster-wise correction was applied using either extent (e) or mass (m, recommended) approaches with  t>2.81 cluster threshold. Clusters were then thresholded at p=0.05. 

In [ ]:
cmd="""#!/bin/bash

module use /projects/ics/modules
module load fsl/6.0.3

umask g+w

export PATH={PATH}/bin:$PATH
cd `dirname {FEATPATH}`

# get gfeat folder for analysis
gfeat=`basename {FEATPATH}`

# to be sure all gfeat models ran to completion
echo $gfeat
featcheck $gfeat

# move mask to working directory 
cp -p /pl/active/banich/masks/wager_masks/bin/bin_wager_gm_mask.nii.gz .

######################################################################
# RUN RANDOMISE
######################################################################
export FSL_SLURM_NUM_CPU=4
export FSL_SLURM_UCB_ACCOUNT={ACCOUNT}
export FSL_SLURM_PARTITION_NAME={PARTITION}
export FSL_SLURM_QUEUE_NAME={QOS}
export FSL_SLURM_WALLTIME_MINUTES=1440
export FSL_SLURM_MB_RAM=16G

. ${{FSLDIR}}/etc/fslconf/fsl.sh

mask=$PWD/bin_wager_gm_mask.nii.gz
cval=2.807034 # for p 0.0025 one-tail
randomise_gfeat_grid3 $gfeat --uncorrp -n 10000 -m $mask -c $cval -C $cval -x

echo Done!
"""

# use template above to run sbatch command to generate mine4stats results
fullpath=os.path.join(path,gfeat_prefix+"*.gfeat")
gfeatdirs = glob.glob(fullpath)
hpc=set_compute_vars()

# writing batch file
for idx,gfeat in enumerate(gfeatdirs):
    with open("rm.randomise"+str(idx)+".sh","w+") as f:
        f.writelines(
            cmd.format(**{
               'PATH':os.getcwd(),  
               'FEATPATH': gfeat,
               'QOS': hpc["qos"], 
               'PARTITION': hpc["partition"], 
               'ACCOUNT': hpc["account"]
            })
        )

    # running batch file
    shell(". ./rm.randomise"+str(idx)+".sh")

!!Wait for all jobs to finish on the queue!!

In [ ]:
# check if job has finished on queue... if no jobs are showing...proceed.
shell('squeue -u $USER')

In [ ]:
# WARNING!! REMOVES FILES BY WILDCARD. BE SURE BEFORE RUNNING!
remove_files_by_wildcard("rm.*.sh")

### STEP 3: RANDOMISE CLUSTER REPORTS
Clusters, corrected for multiple comparisons using either FDR, FWE, voxel, and uncorrected, are recorded. Cluster centers are matched based on binary atlas mapping to the Talairach and Harvard Oxford (Cortical and Subcortical) Structral atalases. Local maxima are defined.

In [ ]:
cmd="""#!/bin/bash
#
#SBATCH --job-name=randomise.clusters
#SBATCH --qos={QOS}
#SBATCH --partition={PARTITION}
#SBATCH --account={ACCOUNT}
#SBATCH --time=24:00:00
#SBATCH --array=1-{NDIRS}
#SBATCH --output=logs/randomise_clusters_%A_%a.out
#SBATCH --error=logs/randomise_clusters_%A_%a.err
#SBATCH --cpus-per-task=4
#SBATCH --mem=16G

umask g+w

module use /projects/ics/modules
module load fsl/6.0.3

export PATH={PATH}/bin:$PATH
cd {FEATPATH}

# get gfeat folder for analysis
gfeat=`ls -d {PREFIX}*.gfeat | sed -n "$SLURM_ARRAY_TASK_ID p"`

######################################################################
# For FDR, specify GM mask instead of default cope*.feat/mask.nii.gz
# with output in *.gfeat/fdrmask_summary_stats
######################################################################
#     fdrmask       = 0.95
#     fwepclustrerm = 0.95
#     fwepvox       = 0.95
#     vox_uncorrp   = 0.999
#
# ========= RUN ANY OF THESE  =========
# ......... FDR
randclustercontrasts10000_fdrmask $PWD/$gfeat $mask

# ......... FWEP CLUSTERM
randclustercontrasts10000_fwep_clusterm $PWD/$gfeat

# ......... FWEP VOX
randclustercontrasts10000_fwep_vox $PWD/$gfeat

# ......... VOX UNCORRP (vox_uncorrp = 0.999)
randclustercontrasts10000_vox_uncorrp $PWD/$gfeat


# After
#    (a) randomise has completed
#    (b) cluster reports are run
# with outputs in*.gfeat/*summary_stats/thresh_tstats

# Run jobs on the grid since the Tal Daemon calls have been replaced
# by references to the FSL Tal Atlas

# -- generate *.gfeat/*summary_stats/localmax based on ../thresh_tstats
# randomise clusters
# ========= RUN ANY OF THESE  =========
# ......... FDR
peakdist=12
n=50
s=$gfeat/fdrmask_summary_stats
echo $s
getlocalmax4 $s $peakdist $n

#
#  ......... FWEP CLUSTERM
peakdist=12
n=50
s=$gfeat/fwepclusterm_summary_stats
echo $s
getlocalmax4 $s $peakdist $n

#
#  ......... FWEP VOX
peakdist=12
n=50
s=$gfeat/fwepvox_summary_stats
echo $s
getlocalmax4 $s $peakdist $n

#
#  ......... VOX UNCORRP
peakdist=12
n=50
s=$gfeat/vox_uncorrp_summary_stats
echo $s
getlocalmax4 $s $peakdist $n


echo Done!
"""

# use template above to run sbatch command to generate mine4stats results
fullpath=os.path.join(path,gfeat_prefix+"*.gfeat")
gfeatdirs = glob.glob(fullpath)
hpc=set_compute_vars()

# writing batch file
with open("rm.randomise.clusters.sh","w+") as f:
    f.writelines(
        cmd.format(**{
           'PATH':os.getcwd(), 
           'NDIRS':len(gfeatdirs), 
           'FEATPATH': path,
           'PREFIX': gfeat_prefix, 
           'QOS': hpc["qos"], 
           'PARTITION': hpc["partition"], 
           'ACCOUNT': hpc["account"]
        })
    )

# running batch file
shell("sbatch rm.randomise.clusters.sh")

In [ ]:
# check if job has finished on queue... if no jobs are showing...proceed.
shell('squeue -u $USER')

!!Wait for jobs to finish on the cluster!!

In [ ]:
# WARNING!! REMOVES FILES BY WILDCARD. BE SURE BEFORE RUNNING!
remove_files_by_wildcard("rm.*.sh")

### STEP 4: MINE4STATS + OVERLAYS
Univariate model results are stored in a summary folder for ease of review. Uncorrected zstats, and if available randomise tstats thresholded using FDR, FWE, TFCE methods. A description of the contrast names and lower model design are also stored. 

In [ ]:
cmd="""#!/bin/bash
#
#SBATCH --job-name=mine4stats
#SBATCH --qos={QOS}
#SBATCH --partition={PARTITION}
#SBATCH --account={ACCOUNT}
#SBATCH --time=00:30:00
#SBATCH --array=1-{NDIRS}
#SBATCH --output=logs/mine4stats_%A_%a.out
#SBATCH --error=logs/mine4stats_%A_%a.err
#SBATCH --cpus-per-task=2
#SBATCH --mem=8G

module use /projects/ics/modules
module load fsl/6.0.7

export PATH={PATH}/bin:$PATH
cd {FEATPATH}
gfeat=`ls -d {PREFIX}*.gfeat | sed -n "$SLURM_ARRAY_TASK_ID p"`
mine4stats_randomise $gfeat -z -x -a -d -r ;

# generate LOWER_DESIGN spreadsheet
sumdir=$(echo $gfeat | sed 's|.gfeat|.summary|')
fsfcheck $sumdir/lower_design.fsf | egrep '(EV|contrast)' | grep -v IRR > $sumdir/LOWER_DESIGN.txt

# overlay - produce static images
getoverlays $sumdir

echo Done!
"""

# use template above to run sbatch command to generate mine4stats results
fullpath=os.path.join(path,gfeat_prefix+"*.gfeat")
gfeatdirs = glob.glob(fullpath)
hpc=set_compute_vars()

# writing batch file
with open("rm.mine4stats.sh","w+") as f:
    f.writelines(
        cmd.format(**{
           'PATH':os.getcwd(), 
           'NDIRS':len(gfeatdirs), 
           'FEATPATH': path,
           'PREFIX': gfeat_prefix, 
           'QOS': hpc["qos"], 
           'PARTITION': hpc["partition"], 
           'ACCOUNT': hpc["account"]
        })
    )

# running batch file
shell("sbatch rm.mine4stats.sh")

In [ ]:
# check if job has finished on queue... if no jobs are showing...proceed.
shell('squeue -u $USER')

!!Wait for jobs to finish on the cluster!!

In [ ]:
# WARNING!! REMOVES FILES BY WILDCARD. BE SURE BEFORE RUNNING!
remove_files_by_wildcard("rm.*.sh")

### STEP 5: ZIP SUMMARY

In [ ]:
# create zip
os.makedirs(os.path.join(path,"zips"),exist_ok=True)
cmd="zip -rq "+os.path.join("zips",gfeat_prefix+"_"+time.strftime("%Y%m%d")+".zip")+" "+os.path.join(gfeat_prefix+"*.summary/{*.png,*.fsf,LOWER_DESIGN.txt,README.txt,zstats,randomise,cluster_reports}")
# print(cmd)
shell(cmd,workdir=path)

### STEP 6: CONTRAST PDF REPORT

In [ ]:
def sort_by_copenum(paths):
    try:
        df = pd.DataFrame()
        df["path"] = sumdirs
        df["copnum"] = df["path"].str.extract(r'cope(\d+)').astype(int)
        df = df.sort_values("copnum")
    except:
        pass
    return df["path"].values

def split_list_to_dict(lst):
    result_dict = {}
    for item in lst:
        key, value = item.split(' ', 1)  # Split on first space
        result_dict[key] = value
    return result_dict

In [ ]:
## add subject specific plots (snr, motion, carpet)
sumdirs = glob.glob(os.path.join(path,gfeat_prefix+"*.summary"))
sumdirs_sorted = sort_by_copenum(sumdirs)

In [ ]:
sumdirs_sorted

In [ ]:
# establish lower, intermediate (if applicable), and higher level
# put contrast names from lower and higher levels into an array
cmd="""#!/bin/bash
module use /projects/ics/modules
module load fsl/6.0.7
export PATH=$PWD/bin:$PATH
SOURCE={SOURCE}
lname=("" `fsfinfo -lcon $SOURCE | sed 's|contrast [0-9 ]* ||' | sed -e 's|[() ]|_|g' -e 's|>|-|g' -e 's|<|-lt-|g' -e 's|&|+|g'`); \
hname=("" `fsfinfo -hcon $SOURCE | sed 's|contrast [0-9 ]* ||' | sed -e 's|[() ]|_|g' -e 's|>|-|g' -e 's|<|-lt-|g' -e 's|&|+|g'`)

fsfinfo -i $SOURCE | head -1 | grep -q '\.gfeat/'; if [ $? -eq 0 ]; then  ### SOURCE IS 3-LEVEL ANALYSIS
hasIntermediate=set
iname=("" `fsfinfo -icon $SOURCE | sed 's|contrast [0-9 ]* ||' | sed -e 's|[() ]|_|g' -e 's|>|-|g' -e 's|<|-lt-|g' -e 's|&|+|'`)
fi

### ASK PERMISSION TO CONTINUE
echo "LOWER LEVEL CONTRASTS:";  n=1; for i in ${{lname[*]}}; do echo $n $i; let n++; done
[ $hasIntermediate ] && echo "INTERMEDIATE LEVEL CONTRASTS:";  n=1; for i in ${{iname[*]}}; do echo $n $i; let n++; done
echo "HIGHER LEVEL CONTRASTS:"; n=1; for i in ${{hname[*]}}; do echo $n $i; let n++; done
"""

# writing batch file
with open("rm.get_design.sh","w+") as f:
    f.writelines(
        cmd.format(**{'SOURCE':os.path.join(sumdirs_sorted[0].replace(".summary",".gfeat"))})
    )
out = shell(". ./rm.get_design.sh")

In [ ]:
# WARNING!! REMOVES FILES BY WILDCARD. BE SURE BEFORE RUNNING!
remove_files_by_wildcard("rm.*")

In [ ]:
# pull cope name and cope number mapping for all levels
lid=out.index("LOWER LEVEL CONTRASTS:")
hid=out.index("HIGHER LEVEL CONTRASTS:")

if "INTERMEDIATE LEVEL CONTRASTS:" in out:
    iid=out.index("INTERMEDIATE LEVEL CONTRASTS:")
    lower = split_list_to_dict(out[lid+1:iid])
    inter = split_list_to_dict(out[iid+1:hid])
    higher = split_list_to_dict(out[hid+1:])
else:
    lower = split_list_to_dict(out[lid+1:hid])
    higher = split_list_to_dict(out[hid+1:])


In [ ]:
lower

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, landscape
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak, NextPageTemplate, PageTemplate, Frame, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.rl_config import defaultPageSize
from reportlab.lib import utils

# initialize report settings
PAGE_HEIGHT=defaultPageSize[1]
styles = getSampleStyleSheet()
Elements=[]
HeaderStyle = styles["Heading1"]
HeaderStyle.alignment = 1
ParaStyle = styles["Normal"]
PreStyle = styles["Code"]

def header(txt, style=HeaderStyle, klass=Paragraph, sep=0.3):
    s = Spacer(0.2*inch, sep*inch)
    Elements.append(s)
    para = klass(txt, style)
    Elements.append(para)

    
def p(txt):
    return header(txt, style=ParaStyle, sep=0.1)


def make_portrait(canvas,doc):
    canvas.setPageSize(letter)

    
def make_landscape(canvas,doc):
    canvas.setPageSize(landscape(letter))
    
    
def build_report():
    doc = SimpleDocTemplate(reportfile,pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)
    
    frame1 = Frame(doc.leftMargin, doc.topMargin,
                doc.width, doc.height,
                leftPadding = 0, rightPadding = 0,
                topPadding = 0, bottomPadding = 0,
                id='frame1')
    
    frame2 = Frame(doc.leftMargin, doc.topMargin,
                doc.height, doc.width,
                leftPadding = 0, rightPadding = 0,
                topPadding = 0, bottomPadding = 0,
                id='frame1')
    ptemplate = PageTemplate(id='portrait',frames =[frame1], onPage=make_portrait)
    ltemplate = PageTemplate(id='landscape',frames =[frame2], onPage=make_landscape)
    
    doc.addPageTemplates([ptemplate, ltemplate])
    
    doc.build(Elements)


def get_image(path, width=1*inch):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    im = Image(path, width=width, height=(width * aspect))
    im.hAlign = 'LEFT'
    return im

    
def add_table(outliers):
    # add table of "outlier values"
    outliers['y'] = outliers['y'].apply(lambda x: round(x, 2))
    t=Table(outliers[["subject", "session","acq","y"]].values.tolist())
    Elements.append(t)    


#### UNCORRECTED ZSTAT REPORT (with INTERMEDIATE COPES)

In [ ]:
Title_line1 = "Intermountain Neuroimaging Consortium".upper()
Title_line2 = "UNIVARIATE ANALYSIS REPORT".upper()
TodayDate = date.today().strftime("%B %d, %Y")
Author = "Created By: "+os.environ["USER"]
url = "<github link>"
ProjectName = gfeat_prefix
reportfile = os.path.join(path,"zips",gfeat_prefix+"_zstats_"+time.strftime("%Y%m%d")+".pdf")

header(Title_line1)
header(Title_line2,sep=0.05)
header(ProjectName, sep=0.05, style=styles["Heading2"])

header(TodayDate, sep=0.1, style=ParaStyle)
header(Author, sep=0.1, style=ParaStyle)
header(url, sep=0.1, style=ParaStyle)
counter=0

p("This work utilized the CUmulus on-premise cloud service at the University of Colorado Boulder. CUmulus is jointly "
  "funded by the National Science Foundation (award OAC-1925766) and the University of Colorado Boulder.") 

p("Data storage supported by the University of Colorado Boulder 'PetaLibrary'")

# Elements.append(NextPageTemplate('landscape'))
Elements.append(PageBreak())

header("UNCORRECTED ZSTATS", style=styles["Heading3"])

for d in sumdirs_sorted:
    if 'inter' in locals():
        copenum = re.search('cope(\d+)',d).group(1)
        copename = lower[copenum]

        header("LOWER LEVEL CONTRAST: "+copenum+": "+copename.replace("_"," ").replace("gt",">"), style=styles["Heading4"])
    
    chart_style = TableStyle([('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                                  ('VALIGN', (0, 0), (-1, -1), 'MIDDLE')])

    image1 = get_image(os.path.join(d, "overlays/render1/ramp2.gif"), width=150)
    image2 = get_image(os.path.join(d, "overlays/render1/ramp.gif"), width=150)

    Elements.append(Table([["zstat: -5                             -2.58","  2.58                                       5"],[image1, image2]],
                         colWidths=[2 * inch, 2 * inch],
                         rowHeights=[0.15 * inch, 0.4 * inch], style=chart_style))
    
    # setup expected naming scheme
    for h in higher.keys():
        header("HIGHER LEVEL CONTRAST: "+higher[h], style=styles["Heading5"])
        if 'inter' in locals():
            for i in inter.keys():
                # find corresponding z-stat image from overlays
                imgpaths = glob.glob(os.path.join(d,"overlays/render1/slices1_"+copename+"_"+inter[i]+"_"+higher[h]+".png"))
                log.debug(higher[h]+"... cope"+str(i)+": "+inter[i].replace("_"," ").replace("gt",">"))
                log.debug("file: %s", imgpaths[0])
                if imgpaths:
                    p(higher[h]+"... cope"+str(i)+": "+inter[i].replace("_"," ").replace("gt",">"))
                    Elements.append(get_image(imgpaths[0], width=500))
        else:
            for l in lower.keys():
                # find corresponding z-stat image from overlays
                imgpaths = glob.glob(os.path.join(d,"overlays/render1/slices1_"+lower[l]+"_"+higher[h]+".png"))
                log.debug(higher[h]+"... cope"+str(l)+": "+lower[l].replace("_"," ").replace("gt",">"))
                log.debug("file: %s", imgpaths[0])
                if imgpaths:
                    p(higher[h]+"... cope"+str(l)+": "+lower[l].replace("_"," ").replace("gt",">"))
                    Elements.append(get_image(imgpaths[0], width=500))
    
    Elements.append(PageBreak())

build_report()
log.info("Report written: %s", reportfile)

#### RANDOMISE CLUSTER CORRECTED REPORT: FWE CLUSTERM

In [ ]:
Title_line1 = "Intermountain Neuroimaging Consortium".upper()
Title_line2 = "UNIVARIATE ANALYSIS REPORT".upper()
TodayDate = date.today().strftime("%B %d, %Y")
Author = "Created By: "+os.environ["USER"]
url = "<github link>"
ProjectName = gfeat_prefix
reportfile = os.path.join(path,"zips",gfeat_prefix+"_fwep_clusterm_tstats_"+time.strftime("%Y%m%d")+".pdf")

header(Title_line1)
header(Title_line2,sep=0.05)
header(ProjectName, sep=0.05, style=styles["Heading2"])

header(TodayDate, sep=0.1, style=ParaStyle)
header(Author, sep=0.1, style=ParaStyle)
header(url, sep=0.1, style=ParaStyle)
counter=0

p("This work utilized the CUmulus on-premise cloud service at the University of Colorado Boulder. CUmulus is jointly "
  "funded by the National Science Foundation (award OAC-1925766) and the University of Colorado Boulder.") 

p("Data storage supported by the University of Colorado Boulder 'PetaLibrary'")

# Elements.append(NextPageTemplate('landscape'))
Elements.append(PageBreak())

header("RANDOMISE: FWE CLUSTER MASS CORRECTED TSTATS", style=styles["Heading3"])

for d in sumdirs_sorted:
    # CLUSTER CORRECTED
    if os.path.exists(os.path.join(d,"randomise")):
        
        if 'inter' in locals():
            copenum = re.search('cope(\d+)',d).group(1)
            copename = lower[copenum]
            header("LOWER LEVEL CONTRAST: "+copenum+": "+copename.replace("_"," ").replace("gt",">"), style=styles["Heading4"])

        chart_style = TableStyle([('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                                      ('VALIGN', (0, 0), (-1, -1), 'MIDDLE')])

        image1 = get_image(os.path.join(d, "overlays/render1/ramp2.gif"), width=150)
        image2 = get_image(os.path.join(d, "overlays/render1/ramp.gif"), width=150)

        Elements.append(Table([["tstat: -5                             -2.58","  2.58                                       5"],[image1, image2]],
                             colWidths=[2 * inch, 2 * inch],
                             rowHeights=[0.15 * inch, 0.4 * inch], style=chart_style))
        
        # setup expected naming scheme
        for h in higher.keys():
            header("HIGHER LEVEL CONTRAST: "+higher[h], style=styles["Heading5"])
            # analyses with intermediate results
            if 'inter' in locals():
                for i in inter.keys():
                    # find corresponding z-stat image from overlays
                    imgpaths = glob.glob(os.path.join(d,"overlays/render1/slices1_"+copename+"_thresh_fwep_clusterm_tstat"+str(h)+"_"+inter[i]+"*.png"))

                    if imgpaths:
                        log.debug(higher[h]+"... cope"+str(i)+": "+inter[i].replace("_"," ").replace("gt",">"))
                        log.debug("file: %s", imgpaths[0])
                        p(higher[h]+"... cope"+str(i)+": "+inter[i].replace("_"," ").replace("gt",">"))
                        Elements.append(get_image(imgpaths[0], width=500))
            else:
                for l in lower.keys():
                    # find corresponding z-stat image from overlays
                    imgpaths = glob.glob(os.path.join(d,"overlays/render1/slices1_thresh_fwep_clusterm_tstat"+str(h)+"_"+lower[l]+"*.png"))

                    if imgpaths:
                        log.debug(higher[h]+"... cope"+str(l)+": "+lower[l].replace("_"," ").replace("gt",">"))
                        log.debug("file: %s", imgpaths[0])
                        p(higher[h]+"... cope"+str(l)+": "+lower[l].replace("_"," ").replace("gt",">"))
                        Elements.append(get_image(imgpaths[0], width=500))
                
        Elements.append(PageBreak())

build_report()
log.info("Report written: %s", reportfile)

#### RANDOMISE CLUSTER CORRECTED REPORT: FDR

In [ ]:
Title_line1 = "Intermountain Neuroimaging Consortium".upper()
Title_line2 = "UNIVARIATE ANALYSIS REPORT".upper()
TodayDate = date.today().strftime("%B %d, %Y")
Author = "Created By: "+os.environ["USER"]
url = "<github link>"
ProjectName = gfeat_prefix
reportfile = os.path.join(path,"zips",gfeat_prefix+"_fdr_tstats_"+time.strftime("%Y%m%d")+".pdf")

header(Title_line1)
header(Title_line2,sep=0.05)
header(ProjectName, sep=0.05, style=styles["Heading2"])

header(TodayDate, sep=0.1, style=ParaStyle)
header(Author, sep=0.1, style=ParaStyle)
header(url, sep=0.1, style=ParaStyle)
counter=0

p("This work utilized the CUmulus on-premise cloud service at the University of Colorado Boulder. CUmulus is jointly "
  "funded by the National Science Foundation (award OAC-1925766) and the University of Colorado Boulder.") 

p("Data storage supported by the University of Colorado Boulder 'PetaLibrary'")

# Elements.append(NextPageTemplate('landscape'))
Elements.append(PageBreak())

header("RANDOMISE: FDR` CORRECTED TSTATS", style=styles["Heading3"])

for d in sumdirs_sorted:
    # CLUSTER CORRECTED
    if os.path.exists(os.path.join(d,"randomise")):
        if 'inter' in locals():
            copenum = re.search('cope(\d+)',d).group(1)
            copename = lower[copenum]
            header("LOWER LEVEL CONTRAST: "+copenum+": "+copename.replace("_"," ").replace("gt",">"), style=styles["Heading4"])

        chart_style = TableStyle([('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                                      ('VALIGN', (0, 0), (-1, -1), 'MIDDLE')])

        image1 = get_image(os.path.join(d, "overlays/render1/ramp2.gif"), width=150)
        image2 = get_image(os.path.join(d, "overlays/render1/ramp.gif"), width=150)

        Elements.append(Table([["tstat: -5                             -2.58","  2.58                                       5"],[image1, image2]],
                             colWidths=[2 * inch, 2 * inch],
                             rowHeights=[0.15 * inch, 0.4 * inch], style=chart_style))
        
        # setup expected naming scheme
        for h in higher.keys():
            header("HIGHER LEVEL CONTRAST: "+higher[h], style=styles["Heading5"])
            if 'inter' in locals():
                for i in inter.keys():
                    # find corresponding z-stat image from overlays
                    imgpaths = glob.glob(os.path.join(d,"overlays/render1/slices1_"+copename+"_thresh_fdr_tstat"+str(h)+"_"+inter[i]+"*.png"))

                    if imgpaths:
                        log.debug(higher[h]+"... cope"+str(i)+": "+inter[i].replace("_"," ").replace("gt",">"))
                        log.debug("file: %s", imgpaths[0])
                        p(higher[h]+"... cope"+str(i)+": "+inter[i].replace("_"," ").replace("gt",">"))
                        Elements.append(get_image(imgpaths[0], width=500))
            else:
                for l in lower.keys():
                    # find corresponding z-stat image from overlays
                    imgpaths = glob.glob(os.path.join(d,"overlays/render1/slices1_thresh_fdr_tstat"+str(h)+"_"+lower[l]+"*.png"))

                    if imgpaths:
                        log.debug(higher[h]+"... cope"+str(l)+": "+lower[l].replace("_"," ").replace("gt",">"))
                        log.debug("file: %s", imgpaths[0])
                        p(higher[h]+"... cope"+str(l)+": "+lower[l].replace("_"," ").replace("gt",">"))
                        Elements.append(get_image(imgpaths[0], width=500))
        Elements.append(PageBreak())

build_report()
log.info("Report written: %s", reportfile)

### STEP 7: UPLOAD SUMMARY TO FLYWHEEL

Using The Flywheel CLI or SDK for the first time?
1. You need to use Flywheel's CLI login process to store your flywheel credentials in: `/home/$USER/.config/flywheel/user.json`
2. Access the flywheel CLI using these [instructions](https://inc-documentation.readthedocs.io/en/latest/cli_basics.html#cli-from-blanca-compute-node)
3. Generate an individualized API key using these [instructions](https://docs.flywheel.io/admin/api_keys/admin_creating_a_user_api_key/)

Thats it! You should now be able to access your Flywheel projects!


In [ ]:
import flywheel
fw = flywheel.Client('')

def get_analysis_files(analysis):
    names=[]
    for f in analysis.files:
        names.append(f.name)
    return names

In [ ]:
project = fw.lookup(flywheel_path)
project = fw.get_project(project.id)

analysis_name = "Univariate Analyses "+time.strftime("%m/%d/%Y %H:%M")
if not fw.analyses.find('label='+analysis_name):
    analysis = project.add_analysis(label=analysis_name)
else:
    analysis = fw.analyses.find_one('label='+analysis_name)

files = glob.glob(os.path.join(path,"zips",gfeat_prefix+"*"))

for f in files:
    if os.path.basename(f) not in get_analysis_files(analysis):
        analysis.upload_output(f)